In [1]:
import json, re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
import time

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def create_link_page_to_download(
    sprzedaz: str = 'sprzedaz', 
    apartament_type: str  = 'mieszkanie', 
    region: str  = 'cala-polska',
    price_min: int = None,
    price_max: int = None,
    limit: str ='36',
    page_counter = 1
):
    
    """
    Function to extract all ads from a given page from OtoDom webservice
    
    Args:
        sprzedaz (str): define if it should be: sprzedaz / wynajem
        apartament_type: type of looked apartament. Possible types: miszkanie/kawalerka/dom/inwestycja/pokoj/dzialka/lokal/haleimagazyny/garaz
        region: cala-polska or one from voivodeship:
            dolnoslaskie
            kujawsko--pomorskie
            lodzkie
            lubelskie
            lubuskie
            malopolskie
            mazowieckie
            opolskie
            podkarpackie
            podlaskie
            pomorskie
            slaskie
            swietokrzyskie
            warminsko--mazurskie
            wielkopolskie
            zachodniopomorskie
        price_min (int): price min of looked apartament Default = None,
        price_max(int) = price max of looked apartament. Default = None,
        limit limit of ads on one page possible: 24, 36 48, 72
        page_counter: define a page number
    Returns:
        page_url (str): url from Oto Dom service for downloading ads details
    """
    
    main_page = 'https://www.otodom.pl/pl/wyniki'
    sprzedaz = sprzedaz
    apartament_type = apartament_type
    region = region
    limit = limit
    price_min = price_min
    price_max = price_max
    last_part_if_url = f'&ownerTypeSingleSelect=ALL&priceMin={price_min}&priceMax={price_max}&by=DEFAULT&direction=DESC&viewType=listing&page={page_counter}'
    
    page_url = f'{main_page}/{sprzedaz}/{apartament_type}/{region}?limit={limit}{last_part_if_url}'
    
    return page_url

In [ ]:
def define_page_to_extract(page_url: str):
    """
    Function to extract all ads from a given page from OtoDom webservice
    Args:
        page_url (str): link to the OtoDom page
    
    Returns:
        soup (bs4.BeautifulSoup): bs4.BeautifulSoup object for future downloading data

    """
    
    page = requests.get(page_url)
    print(page)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    return soup

In [ ]:
def create_list_of_ulrs(soup) -> list:
    """
    Function to extract ftom Oto Dom page links to download data from particular ads
    
    Args:
        soup (bs4.BeautifulSoup): bs4.BeautifulSoup object
    
    Returns:
        list: List of linkt to the ads from given page

    """
    
    list_of_url = []
    for a in soup.find_all('a', {"class": "css-lsw81o e1dfeild2"}, href=True):
        one_url = a['href']
        list_of_url.append(one_url)
        
    return list_of_url

In [ ]:
def extract_olx_data(list_of_url: list) -> pd.DataFrame:
    """
    Function to extrat ads details about homes & flat in otodom website
    
    Args:
        list_of_url (list): List of urls to extract data
    
    Returns:
        pdDataFrame: Data Frame with details about ads from given links

    """
    
    full_df_ads_details = []

    for index, one_url in enumerate(list_of_url[1]):

        one_add_page = []
        page_one_add = []
        soup_one_add = []
        json_data = []
        add_details_df = pd.DataFrame()
        characteristics_df = pd.DataFrame()
        deeper_characteristic_df = pd.DataFrame()
        coordinate_df = pd.DataFrame()
        addres_df = pd.DataFrame()
        output_ond_add_details = pd.DataFrame()

        one_add_page = f'https://www.otodom.pl{one_url}'

        page_one_add = requests.get(one_add_page, timeout=2)
        
        print(index, one_add_page, ':', page_one_add)
        
        soup_one_add = BeautifulSoup(page_one_add.content, 'html.parser')

        soup_one_add.find_all("div", {"class": "css-m97llu e16xl7020"})
        json_data = json.loads(soup_one_add.find('script', type='application/json').text)

        try:
            add_details_df["lang"] = pd.json_normalize(json_data['props']['pageProps'])['lang']
        except KeyError:
            add_details_df["lang"] = 'None'

        try:
            add_details_df["ad_id"] = pd.json_normalize(json_data['props']['pageProps'])['id']
        except KeyError:
            add_details_df["ad_id"] = 'None'

        try:
            add_details_df["url"] = pd.json_normalize(json_data['props']['pageProps'])['relativeUrl']
        except KeyError:
            add_details_df["url"] = 'None'

        try:
            add_details_df["ad_id2"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['id']
        except KeyError:
            add_details_df["ad_id2"] = 'None'

        try:
            add_details_df["market"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['market']
        except KeyError:
            add_details_df["market"] = 'None'

        try:
            add_details_df["publicId"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['publicId']
        except KeyError:
            add_details_df["publicId"] = 'None'

        try:
            add_details_df["slug"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['slug']
        except KeyError:
            add_details_df["slug"] = 'None'

        try:
            add_details_df["advertiserType"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['advertiserType']
        except KeyError:
            add_details_df["advertiserType"] = 'None'

        try:
            add_details_df["createdAt"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['createdAt']
        except KeyError:
            add_details_df["createdAt"] = 'None'

        try:
            add_details_df["modifiedAt"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['modifiedAt']
        except KeyError:
            add_details_df["modifiedAt"] = 'None'

        try:
            add_details_df["description"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['description']
        except KeyError:
            add_details_df["description"] = 'None'

        try:
            add_details_df["title"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['title']
        except KeyError:
            add_details_df["title"] = 'None'

        try:
            add_details_df["full_url"] = pd.json_normalize(json_data['props']['pageProps']['ad'])['url']
        except KeyError:
            add_details_df["full_url"] = 'None'

        try:
            add_details_df["price"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'price'")['value']
        except KeyError:
            add_details_df["price"] = 'None'

        try:
            add_details_df["m"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'm'")['value']
        except KeyError:
            add_details_df["m"] = 'None'

        try:
            add_details_df["price_per_m"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'price_per_m'")['value']
        except KeyError:
            add_details_df["price_per_m"] = 'None'

        try:
            add_details_df["rooms_num"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'rooms_num'")['value']
        except KeyError:
            add_details_df["rooms_num"] = 'None'

        try:
            add_details_df["floor_no"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("label == 'Piętro'")['localizedValue']
        except KeyError:
            add_details_df["floor_no"] = 'None'

        try:
            add_details_df["heating"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'heating'")['value']
        except KeyError:
            add_details_df["heating"] = 'None'

        try:
            add_details_df["building_ownership"] = pd.json_normalize(json_data['props']['pageProps']['ad']['characteristics']).query("key == 'building_ownership'")['value']
        except KeyError:
            add_details_df["building_ownership"] = 'None'

        try:
            add_details_df["city"] = pd.json_normalize(json_data['props']['pageProps']['ad']['target'])['City']
        except KeyError:
            add_details_df["city"] = 'None'

        try:
            add_details_df["City_id"] = pd.json_normalize(json_data['props']['pageProps']['ad']['target'])['City_id']
        except KeyError:
            add_details_df["City_id"] = 'None'

        try:
            add_details_df["Country"] = pd.json_normalize(json_data['props']['pageProps']['ad']['target'])['Country']
        except KeyError:
            add_details_df["Country"] = 'None'

        try:
            add_details_df["MarketType"] = pd.json_normalize(json_data['props']['pageProps']['ad']['target'])['MarketType']
        except KeyError:
            add_details_df["MarketType"] = 'None'

        try:
            add_details_df["PriceRange"] = pd.json_normalize(json_data['props']['pageProps']['ad']['target'])['PriceRange']
        except KeyError:
            add_details_df["PriceRange"] = 'None'

        try:
            add_details_df["latitude"] = pd.json_normalize(json_data['props']['pageProps']['ad']['location']['coordinates'])['latitude']
        except KeyError:
            add_details_df["latitude"] = 'None'

        try:
            add_details_df["longitude"] = pd.json_normalize(json_data['props']['pageProps']['ad']['location']['coordinates'])['longitude']
        except KeyError:
            add_details_df["longitude"] = 'None'

        try:
            add_details_df["district_name"] = pd.json_normalize(json_data['props']['pageProps']['ad']['location']['address'])['district.code']
        except KeyError:
            add_details_df["district_name"] = 'None'

        try:
            add_details_df["district.id"] = pd.json_normalize(json_data['props']['pageProps']['ad']['location']['address'])['district.id']
        except KeyError:
            add_details_df["district.id"] = 'None'

        full_df_ads_details.append(add_details_df)
        
    full_df_ads_details = pd.concat(full_df_ads_details, axis=0)    
    return full_df_ads_details
    

In [ ]:
def download_data(
    sprzedaz: str = 'sprzedaz', 
    apartament_type: str  = 'mieszkanie',
    region: str = 'mazowieckie',
    price_min:int = None,
    price_max: int = None,
    limit: str ='72',
    path_to_save_batch_files: str = None,
    path_to_save_full_files: str = None,
    sys_sleeping: int = None
):
    """
    function for downloading data from Oto Dom Service
    
    Args:
        sprzedaz (str): define if it should be: sprzedaz / wynajem
        apartament_type: type of looked apartament. Possible types: miszkanie/kawalerka/dom/inwestycja/pokoj/dzialka/lokal/haleimagazyny/garaz
        region: cala-polska or one from voivodeship:
            dolnoslaskie
            kujawsko--pomorskie
            lodzkie
            lubelskie
            lubuskie
            malopolskie
            mazowieckie
            opolskie
            podkarpackie
            podlaskie
            pomorskie
            slaskie
            swietokrzyskie
            warminsko--mazurskie
            wielkopolskie
            zachodniopomorskie
        price_min (int): price min of looked apartament Default = None,
        price_max(int) = price max of looked apartament. Default = None,
        limit limit of ads on one page possible: 24, 36 48, 72
        path_to_save_batch_files (str): path for saving batch files. Default is saving file in directory from scrip running
        path_to_save_full_files (str): path for saving fill data set file. Default is saving file in directory from scrip running
        sys_sleeping (int): System sleeping between loops. In seconds

    Returns:
        pd.DataFrame: Data Frame with data about ads from Oto Dom service
    """
    
    full_df_ads_data_details = []
    full_ds = []
    date_of_download = date.today()

    running = True
    page_counter = 1
    while running:

        next_link_to_download = []
        next_soup_page = []
        next_list_of_ads_links = []
        next_ads_details = []

        next_link_to_download = create_link_page_to_download(
            sprzedaz = 'sprzedaz', 
            apartament_type  = 'mieszkanie',
            region = 'mazowieckie',
            price_min = None,
            price_max = 150000,
            limit ='24', 

            page_counter = page_counter)

        print(next_link_to_download)

        # step 2

        next_soup_page = define_page_to_extract(page_url = next_link_to_download)

        # step 3 

        next_list_of_ads_links = create_list_of_ulrs(soup = next_soup_page)

        # step 4
        try:
            next_ads_details = extract_olx_data(list_of_url = next_list_of_ads_links)
            next_ads_details.to_csv(f'{path_to_save_batch_files}/data_part_{page_counter}_{date_of_download}.csv')
        except ValueError:
            running = False
            print('No More Data = stop downloading data')

        full_df_ads_data_details.append(next_ads_details)

        page_counter += 1
        time.sleep(sys_sleeping)

    full_df_ads_data_details = full_df_ads_data_details[ : -1]
    full_ds = pd.concat(full_df_ads_data_details, axis=0)

    full_ds.to_csv(f'{path_to_save_full_files}/full_oto_dom_data_{date_of_download}.csv')
    
    return full_ds

In [ ]:
test = download_data(    
    sprzedaz ='sprzedaz', 
    apartament_type  = 'mieszkanie',
    region ='mazowieckie',
    price_min = None,
    price_max = 150000,
    limit = '72',
    path_to_save_batch_files = None,
    path_to_save_full_files = None,
    sys_sleeping = 1)

In [3]:
url = "https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG"

In [4]:
page_one_add = requests.get(url, timeout=2)

soup_one_add = BeautifulSoup(page_one_add.content, 'html.parser')

soup_one_add.find_all("div", {"class": "css-m97llu e16xl7020"})
json_data = json.loads(soup_one_add.find('script', type='application/json').text)

In [6]:
type(json_data)

dict

In [9]:
json_data['props']['pageProps']['lang']

'pl'

In [16]:
json_data.get('props').get('pageProps').get('lang')

'pl'

In [ ]:
json_data['props']['pageProps']['ad']['characteristics']).query("key == 'price'").reset_index()['value']

In [34]:
json_data.get('props').get('pageProps').get('ad').get('characteristics')[1].get('value')

'54.08'

In [47]:
json_data['props']['pageProps']['ad']['characteristics'][5]['localizedValue'] 

'5'

In [51]:
json_data2 = json_data['props']['pageProps']

In [58]:
import yaml

In [55]:
parameters_path = "./config/download_parameters.yaml"
scraper = "./config/scraper.yaml"

In [56]:
def read_config(config_path: str):
    with open(config_path, "r") as file:
        yaml_content = yaml.safe_load(file)
    return yaml_content

In [140]:
scraper_config = read_config(scraper)

In [105]:
lang = json_data2.get(scraper_config["ad_attributes"]["lang"][0])
ad_id = json_data2.get(scraper_config["ad_attributes"]["ad_id"][0])
relative_url = json_data2.get(scraper_config["ad_attributes"]["relativeUrl"][0])

In [107]:
print(lang)
print(ad_id)
print(relative_url)

pl
4nOyG
/pl/oferta/2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG


In [134]:
scraper_config["ad_attributes"]["ad_level"]

['ad', 'id', 'publicId']

In [139]:
json_data2.get(scraper_config["ad_attributes"]["ad_level"][0])

{'id': 64781238,
 'market': 'PRIMARY',
 'services': [],
 'publicId': '4nOyG',
 'slug': '2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG',
 'advertiserType': 'business',
 'advertType': 'DEVELOPER_UNIT',
 'source': 'urn:partner:obidointegration',
 'createdAt': '2023-11-06T13:58:55+01:00',
 'modifiedAt': '2024-01-20T17:37:11+01:00',
 'description': '<ul><li>2-pokojowe mieszkanie<strong> numer C.06.04</strong>  na 5. piętrze w Inwestycji NOW. Dowborczyków 18 Dewelopera Okam Capital</li><li>Ponadstandardowa wysokość pomieszczeń</li><li>Park im. Henryka Sienkiewicza (8 minut spacerem)</li><li>Park Źródliska (10 minut spacerem)</li><li>Przystanek tramwajowy i autobusowy (4 minuty spacerem)</li><li>Stacja PKP "Łódź Fabryczna" (10 minut spacerem)</li><li>Biedronka (9 minut spacerem), Lidl (3 minuty samochodem)</li><li>Galeria Łódzka (5 minut samochodem)</li><li>ul. Piotrkowska (13 minut spacerem)</li></ul><br/><strong>NOW. Dowborczyków 18 - poznaj inwestycję</strong><br/><p>NOW. Dowborczyków 18 to

In [141]:
json_data2.get(scraper_config["ad_attributes"]["ad_level"][0]).get(scraper_config["ad_attributes"]["ad_level"][11])

['teren zamknięty', 'garaż/miejsce parkingowe', 'balkon', 'winda']

In [95]:

for attr_name, attr_path in dupa['ad_attributes'].items():
        for attr_path_step in attr_path:
            # print(attr_path_step)
            print(json_data2.get(attr_path_step))
    #         attr_val = attr_val.get(attr_path_step, {})
    # # json_data2.get(attr_path)
    # print(attr_name, attr_path)

pl
4nOyG
/pl/oferta/2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG
{'id': 64781238, 'market': 'PRIMARY', 'services': [], 'publicId': '4nOyG', 'slug': '2-pokojowe-mieszkanie-54m2-2-balkony-ID4nOyG', 'advertiserType': 'business', 'advertType': 'DEVELOPER_UNIT', 'source': 'urn:partner:obidointegration', 'createdAt': '2023-11-06T13:58:55+01:00', 'modifiedAt': '2024-01-20T17:37:11+01:00', 'description': '<ul><li>2-pokojowe mieszkanie<strong> numer C.06.04</strong>  na 5. piętrze w Inwestycji NOW. Dowborczyków 18 Dewelopera Okam Capital</li><li>Ponadstandardowa wysokość pomieszczeń</li><li>Park im. Henryka Sienkiewicza (8 minut spacerem)</li><li>Park Źródliska (10 minut spacerem)</li><li>Przystanek tramwajowy i autobusowy (4 minuty spacerem)</li><li>Stacja PKP "Łódź Fabryczna" (10 minut spacerem)</li><li>Biedronka (9 minut spacerem), Lidl (3 minuty samochodem)</li><li>Galeria Łódzka (5 minut samochodem)</li><li>ul. Piotrkowska (13 minut spacerem)</li></ul><br/><strong>NOW. Dowborczyków 18 - po

In [89]:
dupa['ad_attributes']['lang'][0]

'lang'

In [81]:
char_map = json_data2.get("ad", {}).get("characteristics", {})

offer_characteristics = {
    c["key"]: c["value"]
    for c in char_map
    if c["key"] in dupa['characteristics']
}
offer_parsed.update(offer_characteristics)

In [82]:
offer_parsed

{'price': '',
 'm': '54.08',
 'price_per_m': '',
 'rooms_num': '2',
 'floor_no': 'floor_5',
 'building_ownership': 'full_ownership'}